In [1]:
import cv2
import torch
import numpy as np


In [2]:
import torch.nn as nn
import torch.nn.functional as F
from utils.tensorboard import TensorBoard
from Renderer.model import FCN
from Renderer.stroke_gen import *

In [3]:
writer = TensorBoard("../train_log/line")
import torch.optim as optim

criterion = nn.MSELoss()
net = FCN()
optimizer = optim.Adam(net.parameters(), lr=3e-6)
batch_size = 64

use_cuda = torch.cuda.is_available()
step = 0

In [4]:
def save_model():
    if use_cuda:
        net.cpu()
    torch.save(net.state_dict(), "./model/renderer_line.pkl")
    if use_cuda:
        net.cuda()


def load_weights():
    pretrained_dict = torch.load("./model/renderer_line.pkl")
    model_dict = net.state_dict()
    pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict}
    model_dict.update(pretrained_dict)
    net.load_state_dict(model_dict)


# load_weights()
# 500000
while step < 50000:
    net.train()
    train_batch = []
    ground_truth = []
    for i in range(batch_size):
        f = np.random.uniform(0, 1, 10)
        train_batch.append(f)
        ground_truth.append(draw_line(f))

    train_batch = torch.tensor(train_batch).float()
    ground_truth = torch.tensor(ground_truth).float()
    if use_cuda:
        net = net.cuda()
        train_batch = train_batch.cuda()
        ground_truth = ground_truth.cuda()
    gen = net(train_batch)
    optimizer.zero_grad()
    loss = criterion(gen, ground_truth)
    loss.backward()
    optimizer.step()
    print(step, loss.item())
#     if step < 200000:
#         lr = 1e-4
#     elif step < 400000:
#         lr = 1e-5
#     else:
#         lr = 1e-6
    if step < 20000:
        lr = 1e-4
    elif step < 40000:
        lr = 1e-5
    else:
        lr = 1e-6
    for param_group in optimizer.param_groups:
        param_group["lr"] = lr
    writer.add_scalar("train_line/loss", loss.item(), step)
    if step % 10 == 0:
        net.eval()
        gen = net(train_batch)
        loss = criterion(gen, ground_truth)
        writer.add_scalar("val_line/loss", loss.item(), step)
        for i in range(32):
            G = gen[i].cpu().data.numpy()
            GT = ground_truth[i].cpu().data.numpy()
            writer.add_image("train_line/gen{}.png".format(i), G, step)
            writer.add_image("train_line/ground_truth{}.png".format(i), GT, step)
    if step % 100 == 0:
        save_model()
    step += 1

0 0.2438991218805313
1 0.24326586723327637
2 0.24452805519104004
3 0.24256563186645508
4 0.24288266897201538
5 0.24354898929595947
6 0.2430487722158432
7 0.2425508052110672
8 0.2407420575618744
9 0.2436232566833496
10 0.2413865327835083
11 0.24186231195926666
12 0.24231503903865814
13 0.24091561138629913
14 0.24226075410842896
15 0.24245001375675201
16 0.24250201880931854
17 0.24261853098869324
18 0.24166353046894073
19 0.24092066287994385
20 0.24215343594551086
21 0.24198086559772491
22 0.24189838767051697
23 0.2419675886631012
24 0.24238577485084534
25 0.2416156530380249
26 0.2426667958498001
27 0.24193771183490753
28 0.2409072369337082
29 0.24227875471115112
30 0.24148692190647125
31 0.24174785614013672
32 0.24197016656398773
33 0.24139735102653503
34 0.24174517393112183
35 0.24056476354599
36 0.2407519668340683
37 0.2407778650522232
38 0.24055743217468262
39 0.24020588397979736
40 0.24003224074840546
41 0.2412860095500946
42 0.2417803406715393
43 0.24093182384967804
44 0.2412313967

350 0.07547197490930557
351 0.07744883745908737
352 0.0777546763420105
353 0.07551442831754684
354 0.08610616624355316
355 0.07109525799751282
356 0.08694382756948471
357 0.07448706030845642
358 0.0730818659067154
359 0.06822244077920914
360 0.07844145596027374
361 0.08034788817167282
362 0.07517362385988235
363 0.07220309972763062
364 0.0711640864610672
365 0.07907190173864365
366 0.06396494805812836
367 0.07912330329418182
368 0.07142040133476257
369 0.07502525299787521
370 0.0758104920387268
371 0.08275248110294342
372 0.06969758123159409
373 0.07277581095695496
374 0.06476232409477234
375 0.06543165445327759
376 0.0625169649720192
377 0.07426930963993073
378 0.06600672006607056
379 0.07778751105070114
380 0.06806203722953796
381 0.06779910624027252
382 0.07975517958402634
383 0.06237149238586426
384 0.07586569339036942
385 0.06460945308208466
386 0.06481436640024185
387 0.07734284549951553
388 0.07829749584197998
389 0.0629320964217186
390 0.06640629470348358
391 0.0652380734682083

691 0.044755853712558746
692 0.056994445621967316
693 0.052958548069000244
694 0.05501009523868561
695 0.06265447288751602
696 0.05036694183945656
697 0.05200989916920662
698 0.04361267387866974
699 0.0527506098151207
700 0.05048469454050064
701 0.04902465641498566
702 0.05208027735352516
703 0.05388197675347328
704 0.04654855281114578
705 0.049506776034832
706 0.052671678364276886
707 0.05904671549797058
708 0.0514918752014637
709 0.046864259988069534
710 0.054600801318883896
711 0.0565936453640461
712 0.04507169872522354
713 0.05384897068142891
714 0.04829768091440201
715 0.047246430069208145
716 0.04604101926088333
717 0.0521649532020092
718 0.0534309521317482
719 0.053083207458257675
720 0.05264657735824585
721 0.05250580981373787
722 0.05318306386470795
723 0.048132866621017456
724 0.05665559321641922
725 0.04884549602866173
726 0.04811493307352066
727 0.04988737404346466
728 0.05091499164700508
729 0.05340753123164177
730 0.04943643510341644
731 0.05396604910492897
732 0.05486472

1028 0.04555659368634224
1029 0.048395801335573196
1030 0.03817089647054672
1031 0.039458196610212326
1032 0.044600263237953186
1033 0.04073835164308548
1034 0.04408949241042137
1035 0.04441811516880989
1036 0.05217093601822853
1037 0.04827522113919258
1038 0.04034759849309921
1039 0.0402597077190876
1040 0.04135378822684288
1041 0.046620551496744156
1042 0.04714561626315117
1043 0.037906426936388016
1044 0.04446454346179962
1045 0.046926360577344894
1046 0.04068010300397873
1047 0.04293049871921539
1048 0.04510580375790596
1049 0.04455830529332161
1050 0.039469555020332336
1051 0.04633277654647827
1052 0.04209485277533531
1053 0.038203444331884384
1054 0.04053351283073425
1055 0.044321704655885696
1056 0.04152112081646919
1057 0.039929598569869995
1058 0.04585544392466545
1059 0.043745238333940506
1060 0.04257461801171303
1061 0.04413166269659996
1062 0.044659681618213654
1063 0.0471024364233017
1064 0.03517627343535423
1065 0.040900081396102905
1066 0.044290464371442795
1067 0.045551

1353 0.03282066434621811
1354 0.03795311599969864
1355 0.037279415875673294
1356 0.034367360174655914
1357 0.03259280323982239
1358 0.039090100675821304
1359 0.03591335937380791
1360 0.034302424639463425
1361 0.03743185102939606
1362 0.03537778556346893
1363 0.03466648980975151
1364 0.03670091927051544
1365 0.03850257024168968
1366 0.03419455885887146
1367 0.03623640537261963
1368 0.034293677657842636
1369 0.02928391471505165
1370 0.03604331612586975
1371 0.03258126601576805
1372 0.03489983454346657
1373 0.034643251448869705
1374 0.03341950848698616
1375 0.036603864282369614
1376 0.03227861598134041
1377 0.03343428671360016
1378 0.033352337777614594
1379 0.027160361409187317
1380 0.034122176468372345
1381 0.03624255210161209
1382 0.035170778632164
1383 0.03461106866598129
1384 0.03435068577528
1385 0.03514734283089638
1386 0.03331931307911873
1387 0.033918656408786774
1388 0.03260251134634018
1389 0.03414039686322212
1390 0.02843412011861801
1391 0.03471626713871956
1392 0.039218094199

...

48507 0.005682827439159155
48508 0.005047901999205351
48509 0.004685030784457922
48510 0.005590244196355343
48511 0.005664268974214792
48512 0.005220654420554638
48513 0.005824384745210409
48514 0.005700324662029743
48515 0.006808955688029528
48516 0.005205798428505659
48517 0.005967386066913605
48518 0.007199348881840706
48519 0.004781301598995924
48520 0.004890710115432739
48521 0.006029287353157997
48522 0.00429228600114584
48523 0.004791752900928259
48524 0.005995835177600384
48525 0.0061161029152572155
48526 0.005807294975966215
48527 0.005523210857063532
48528 0.006581920199096203
48529 0.004462914075702429
48530 0.00549223693087697
48531 0.005385449156165123
48532 0.005257722456008196
48533 0.005679920315742493
48534 0.005673691164702177
48535 0.004404569510370493
48536 0.005208292044699192
48537 0.006644667126238346
48538 0.006324982736259699
48539 0.006045655347406864
48540 0.004651785362511873
48541 0.005815817043185234
48542 0.005529217887669802
48543 0.004927871748805046
48

48811 0.004832653794437647
48812 0.006028933450579643
48813 0.005716059356927872
48814 0.005823921412229538
48815 0.005912317428737879
48816 0.004789529833942652
48817 0.005569407716393471
48818 0.006527166347950697
48819 0.0055070542730391026
48820 0.0043081436306238174
48821 0.0063239773735404015
48822 0.004464394878596067
48823 0.005803917534649372
48824 0.005701778922230005
48825 0.004248793236911297
48826 0.0048118168488144875
48827 0.005406390875577927
48828 0.004390460439026356
48829 0.005462722387164831
48830 0.005030995234847069
48831 0.005185441114008427
48832 0.004881367087364197
48833 0.005541082005947828
48834 0.004456649534404278
48835 0.005055236164480448
48836 0.005491036921739578
48837 0.005498291924595833
48838 0.005657650530338287
48839 0.005731561686843634
48840 0.005721851717680693
48841 0.005364474840462208
48842 0.004745557904243469
48843 0.00456406082957983
48844 0.004628439899533987
48845 0.005224277731031179
48846 0.005637113004922867
48847 0.00517696514725685

49114 0.005969508551061153
49115 0.005546867847442627
49116 0.005047003738582134
49117 0.004734727554023266
49118 0.005045874044299126
49119 0.005418794695287943
49120 0.005648092832416296
49121 0.005840818863362074
49122 0.006341584026813507
49123 0.004531617742031813
49124 0.005246057640761137
49125 0.005969159305095673
49126 0.00589667959138751
49127 0.0052663590759038925
49128 0.005255562718957663
49129 0.005563589744269848
49130 0.0053206514567136765
49131 0.004980025347322226
49132 0.005190771073102951
49133 0.005561131983995438
49134 0.005036800168454647
49135 0.005121628753840923
49136 0.005708271637558937
49137 0.004870545119047165
49138 0.00510063162073493
49139 0.005490095820277929
49140 0.00702781043946743
49141 0.004934303928166628
49142 0.005502619314938784
49143 0.004907330963760614
49144 0.005705804564058781
49145 0.0056494055315852165
49146 0.005706828087568283
49147 0.0056052254512906075
49148 0.005263505503535271
49149 0.006376767996698618
49150 0.00454200804233551
4

49417 0.005188619252294302
49418 0.005279616452753544
49419 0.0056332251988351345
49420 0.005030137021094561
49421 0.004720053169876337
49422 0.005584072787314653
49423 0.0038179371040314436
49424 0.005172626115381718
49425 0.006162787321954966
49426 0.0054737054742872715
49427 0.004930941388010979
49428 0.005306079518049955
49429 0.004860591143369675
49430 0.004977087024599314
49431 0.005114712752401829
49432 0.005268921609967947
49433 0.005087294615805149
49434 0.004309540614485741
49435 0.005564258433878422
49436 0.00609250832349062
49437 0.005608334671705961
49438 0.004882178734987974
49439 0.0042373850010335445
49440 0.006484408862888813
49441 0.004968723282217979
49442 0.006121600046753883
49443 0.005228710360825062
49444 0.0048046656884253025
49445 0.005109546240419149
49446 0.0048749325796961784
49447 0.006599553860723972
49448 0.005480890162289143
49449 0.005319687072187662
49450 0.006696777418255806
49451 0.004464712925255299
49452 0.005234350450336933
49453 0.005634726956486

49720 0.004579543136060238
49721 0.004966282285749912
49722 0.005588756408542395
49723 0.004792040213942528
49724 0.006335187703371048
49725 0.005765056237578392
49726 0.005325426813215017
49727 0.006326847709715366
49728 0.004987579770386219
49729 0.005238968413323164
49730 0.0041869874112308025
49731 0.005799978505820036
49732 0.00512111559510231
49733 0.005520414561033249
49734 0.003735243109986186
49735 0.005800403654575348
49736 0.005403261631727219
49737 0.005061543080955744
49738 0.005292140878736973
49739 0.004853882826864719
49740 0.006331465672701597
49741 0.003939972259104252
49742 0.006490837316960096
49743 0.00548152020201087
49744 0.00411613006144762
49745 0.005104718264192343
49746 0.005120535381138325
49747 0.006333976984024048
49748 0.006565587595105171
49749 0.0059431204572319984
49750 0.004317286890000105
49751 0.004463315010070801
49752 0.0045206076465547085
49753 0.00530057679861784
49754 0.006827407516539097
49755 0.005513140000402927
49756 0.005168682895600796
49